In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn as sk

import acquire
import prepare

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.precision', 2)

# Titanic Logistic Regression

In [2]:
df = acquire.get_titanic_data()
df = prepare.prep_titanic(df)

Reading from local CSV...


In [3]:
target = 'survived'
positive = 1

In [4]:
# create empty dataframe to store model results
model_results = pd.DataFrame(columns=['model_number', 'metric_type', 'sample_type', 'score'])

# empty dataframe to store information about the model itself
model_info = pd.DataFrame(columns=['model_number', 'features'])

In [5]:
# split the samples
train, test, validate = prepare.train_test_validate_split(df, target)

train	 n = 399
test	 n = 143
validate n = 172


In [6]:
# establish baseline predictions and create dataframe to calculate performance
train_results = pd.DataFrame()
train_results['actual'] = train[target]
train_results['baseline'] = train[target].mode()[0]

In [7]:
# separate each sample into x and y
x_train = train.drop(columns=target)
y_train = train[target]

x_validate = validate.drop(columns=target)
y_validate = validate[target]

x_test = test.drop(columns=target)
y_test = test[target]

In [8]:
# store baseline metrics

model_number = 'baseline'

# store info about the model
dct = {'model_number': model_number,
       'features': 'N/A',
       'C_value': 'N/A'}
model_info = model_info.append(dct, ignore_index=True)

# establish baseline predictions for train sample
y_pred = baseline_pred = pd.Series([train[target].mode()[0]]).repeat(len(train))

# get metrics
dct = {'model_number': model_number, 
       'sample_type': 'train', 
       'metric_type': 'accuracy',
       'score': sk.metrics.accuracy_score(y_train, y_pred)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'train', 
       'metric_type': 'precision',
       'score': sk.metrics.precision_score(y_train, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'train', 
       'metric_type': 'recall',
       'score': sk.metrics.recall_score(y_train, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'train', 
       'metric_type': 'f1_score',
       'score': sk.metrics.f1_score(y_train, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

# establish baseline predictions for validate sample
y_pred = baseline_pred = pd.Series([train[target].mode()[0]]).repeat(len(validate))

# get metrics
dct = {'model_number': model_number, 
       'sample_type': 'validate', 
       'metric_type': 'f1_score',
       'score': sk.metrics.f1_score(y_validate, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'validate', 
       'metric_type': 'accuracy',
       'score': sk.metrics.accuracy_score(y_validate, y_pred)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'validate', 
       'metric_type': 'precision',
       'score': sk.metrics.precision_score(y_validate, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': 'validate', 
       'metric_type': 'recall',
       'score': sk.metrics.recall_score(y_validate, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

In [9]:
train.head()

,survived,pclass,age,n_sibs_and_spouse,n_parents_and_children,fare,alone,family_size,sex_male,embark_town_Queenstown,embark_town_Southampton
312,0,2,26.0,1,1,26.00,0,2,0,0,1
376,1,3,22.0,0,0,7.25,1,0,0,0,1
41,0,2,27.0,1,0,21.00,0,1,0,0,1
278,0,3,7.0,4,1,29.12,0,5,1,1,0
675,0,3,18.0,0,0,7.78,1,0,1,0,1


### 1. Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline?

This model performs about 9-12% better in accuracy than the baseline. 

### 2. Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

Adding sex to the model improves accuracy by an additional 6-14%. 

### 3. Try out other combinations of features and models.


### 4. Use your best 3 models to predict and evaluate on your validate sample.


In [10]:
def display_model_results():
    return model_results.pivot_table(columns='model_number', 
                                     index=('metric_type', 'sample_type'), 
                                     values='score',
                                     aggfunc=lambda x: x)

In [11]:
features1 = ['age', 'fare', 'pclass']
features2 = ['age', 'fare', 'pclass', 'sex_male']
features3 = ['age', 'fare', 'pclass', 'sex_male', 'alone']
features4 = ['age', 'fare', 'pclass', 'sex_male', 'family_size']
feature_combos = [features1, features2, features3, features4]
C_values = [10, 1, .1]
model_number = 0


for features in feature_combos:
    for C in C_values:

                
        model_number += 1
        
        # store info about the model
        dct = {'model_number': model_number,
               'features': features,
               'C_value': C}
        model_info = model_info.append(dct, ignore_index=True)
        
        # separate each sample into x and y
        x_train = train.drop(columns=target)
        x_train = x_train[features]
        y_train = train[target]

        x_validate = validate.drop(columns=target)
        x_validate = x_validate[features]
        y_validate = validate[target]
        
        
        # create the classifer
        clf = LogisticRegression(C=C, random_state=42)

        # fit the classifier to the training data
        clf = clf.fit(x_train, y_train)

        
        # results for train sample
        y_pred = clf.predict(x_train)
    
                
        # get metrics
        dct = {'model_number': model_number, 
               'sample_type': 'train', 
               'metric_type': 'accuracy',
               'score': sk.metrics.accuracy_score(y_train, y_pred)}
        model_results = model_results.append(dct, ignore_index=True)
        
        dct = {'model_number': model_number, 
               'sample_type': 'train', 
               'metric_type': 'precision',
               'score': sk.metrics.precision_score(y_train, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)

        dct = {'model_number': model_number, 
               'sample_type': 'train', 
               'metric_type': 'recall',
               'score': sk.metrics.recall_score(y_train, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)

        dct = {'model_number': model_number, 
               'sample_type': 'train', 
               'metric_type': 'f1_score',
               'score': sk.metrics.f1_score(y_train, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)
        
        
        # results for validate sample
        x_train
        y_pred = clf.predict(x_validate)

        # get metrics
        dct = {'model_number': model_number, 
               'sample_type': 'validate', 
               'metric_type': 'f1_score',
               'score': sk.metrics.f1_score(y_validate, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)

        dct = {'model_number': model_number, 
               'sample_type': 'validate', 
               'metric_type': 'accuracy',
               'score': sk.metrics.accuracy_score(y_validate, y_pred)}
        model_results = model_results.append(dct, ignore_index=True)

        dct = {'model_number': model_number, 
               'sample_type': 'validate', 
               'metric_type': 'precision',
               'score': sk.metrics.precision_score(y_validate, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)

        dct = {'model_number': model_number, 
               'sample_type': 'validate', 
               'metric_type': 'recall',
               'score': sk.metrics.recall_score(y_validate, y_pred, pos_label=positive)}
        model_results = model_results.append(dct, ignore_index=True)     

In [12]:
model_info

,model_number,features,C_value
0,baseline,N/A,N/A
1,1,"[age, fare, pclass]",10
2,2,"[age, fare, pclass]",1
3,3,"[age, fare, pclass]",0.1
4,4,"[age, fare, pclass, sex_male]",10
5,5,"[age, fare, pclass, sex_male]",1
6,6,"[age, fare, pclass, sex_male]",0.1
7,7,"[age, fare, pclass, sex_male, alone]",10
8,8,"[age, fare, pclass, sex_male, alone]",1
9,9,"[age, fare, pclass, sex_male, alone]",0.1


In [13]:
display_model_results()

model_number                1     2     3     4     5     6     7     8     9  \
metric_type sample_type                                                         
accuracy    train        0.71  0.71  0.73  0.80  0.80  0.79  0.80  0.80  0.80   
            validate     0.68  0.68  0.66  0.80  0.81  0.80  0.81  0.82  0.81   
f1_score    train        0.58  0.58  0.60  0.75  0.74  0.72  0.75  0.74  0.73   
            validate     0.55  0.55  0.51  0.75  0.76  0.74  0.76  0.78  0.75   
precision   train        0.70  0.71  0.78  0.77  0.78  0.81  0.77  0.78  0.81   
            validate     0.64  0.64  0.61  0.76  0.77  0.80  0.77  0.78  0.80   
recall      train        0.49  0.49  0.48  0.72  0.71  0.65  0.72  0.71  0.66   
            validate     0.49  0.49  0.44  0.74  0.76  0.69  0.76  0.77  0.70   

model_number               10    11    12  baseline  
metric_type sample_type                              
accuracy    train        0.80  0.80  0.80      0.59  
            validate     0.78  0.78  0.79      0.59  
f1_score    train        0.74  0.74  0.73      0.00  
            validate     0.72  0.71  0.73      0.00  
precision   train        0.78  0.78  0.80      0.00  
            validate     0.76  0.76  0.77      0.00  
recall      train        0.71  0.71  0.67      0.00  
            validate     0.69  0.67  0.69      0.00

### 5. Choose your best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?


Model # 9 has the best accuracy score, so we will apply it to the test dataset. 

In [14]:
model_number = 9
sample_type = 'test'
features = ['age', 'fare', 'pclass', 'sex_male', 'alone']
C = .1

x_test = test.drop(columns=target)
x_test = test[features]
y_test = test[target]

# create the classifer
clf = LogisticRegression(C=C, random_state=42)

# fit the classifier to the training data
clf = clf.fit(x_train, y_train)

# results for test sample
y_pred = clf.predict(x_test)

# get metrics
dct = {'model_number': model_number, 
       'sample_type': sample_type, 
       'metric_type': 'accuracy',
       'score': sk.metrics.accuracy_score(y_test, y_pred)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': sample_type, 
       'metric_type': 'precision',
       'score': sk.metrics.precision_score(y_test, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': sample_type, 
       'metric_type': 'recall',
       'score': sk.metrics.recall_score(y_test, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

dct = {'model_number': model_number, 
       'sample_type': sample_type, 
       'metric_type': 'f1_score',
       'score': sk.metrics.f1_score(y_test, y_pred, pos_label=positive)}
model_results = model_results.append(dct, ignore_index=True)

In [15]:
model_results[(model_results.metric_type == 'accuracy') & (model_results.model_number == 9)]

,model_number,metric_type,sample_type,score
72,9,accuracy,train,0.80
77,9,accuracy,validate,0.81
104,9,accuracy,test,0.76


We see a dropoff in accuracy score from 81% on the validate sample to 76% on the test sample. This is still 17% better than the baseline accuracy of 59%. 

In [16]:
model_results

,model_number,metric_type,sample_type,score
0,baseline,accuracy,train,0.59
1,baseline,precision,train,0.00
2,baseline,recall,train,0.00
3,baseline,f1_score,train,0.00
4,baseline,f1_score,validate,0.00
...,...,...,...,...
103,12,recall,validate,0.69
104,9,accuracy,test,0.76
105,9,precision,test,0.71
106,9,recall,test,0.69
